# Summary source prediction: Embedded models

Sébastien Meyer

In [1]:
import json
import re
import string

from tqdm import tqdm

import numpy as np
import pandas as pd

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

from xgboost import XGBClassifier

import gensim
from gensim.models import word2vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.utils import tokenize, effective_n_jobs
from gensim.test.utils import common_texts, get_tmpfile

from keras.preprocessing import sequence, text
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import (
    BatchNormalization, Bidirectional, Conv1D, Flatten, GlobalMaxPooling1D, MaxPooling1D, SpatialDropout1D
)
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU, LSTM
from keras.callbacks import EarlyStopping

from tensorflow.keras import preprocessing as kprocessing
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models

import transformers

import matplotlib.pyplot as plt
import seaborn as sns


from src.preprocessing.features.embeddings import text_to_tokens, text_to_vec

In [2]:
stopwords = nltk.corpus.stopwords.words("english")
punct = string.punctuation.replace("-", "")

tqdm.pandas()

## Dictionary and embeddings

In [3]:
train_df = pd.read_json("data/train_set.json")
test_df = pd.read_json("data/test_set.json")
documents = pd.read_json("data/documents.json")

In [4]:
print("Cleaning documents...")

train_df["document_token"] = train_df["document"].progress_apply(
    lambda x: text_to_tokens(x, stopwords, punct, remove_stopwords=True)
)

documents["document_token"] = documents["document"].progress_apply(
    lambda x: text_to_tokens(x, stopwords, punct, remove_stopwords=True)
)

print("All documents clean.")

Cleaning documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [01:17<00:00, 643.28it/s]

All documents clean.


In [5]:
all_docs = train_df["document_token"].to_list() + documents["document_token"].to_list()

## Choose your embeddings...

In [6]:
# Pretrained GloVe embeddings
embeddings_index = {}

f = open("data/embed/glove_300.txt", encoding="utf8")

for line in tqdm(f):
    
    values = line.strip().split(" ")
    try:
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
    except ValueError:
        print(values[0])
    embeddings_index[word] = coefs
    
f.close()

print(f"Found {len(embeddings_index)} word vectors.")

2196017it [01:39, 22074.02it/s]

Found 2196007 word vectors.


In [7]:
# Pretrained Google Word2Vec embeddings
# embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(
#     "data/embed/google_300.gz", binary=True
# )

In [8]:
# Train your own Word2Vec embeddings
class callback(CallbackAny2Vec):
    """Callback to print loss after each epoch."""
    def __init__(self):
        
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        
        total_loss = model.get_latest_training_loss()
        current_loss = total_loss - self.loss_to_be_subed
        self.loss_to_be_subed = total_loss
        
        print(f"Loss after epoch {self.epoch}: {current_loss}")
        
        self.epoch += 1

# w2v = word2vec.Word2Vec(
#     all_docs, vector_size=300, window=20, min_count=5, workers=effective_n_jobs(-1), epochs=25,
#     compute_loss=True, callbacks=[callback()]
# )

# embeddings_index = w2v.wv

## Train test split and transf.

In [9]:
df_train, df_val = train_test_split(train_df, test_size=0.2, random_state=42)

x_train = df_train["summary"].to_numpy()
y_train = df_train["label"].to_numpy().flatten()
x_val = df_val["summary"].to_numpy()
y_val = df_val["label"].to_numpy()

In [10]:
# create sentence vectors using the above function for training and validation set
x_train_glove = [text_to_vec(embeddings_index, x, stopwords, punct, remove_stopwords=True) for x in tqdm(x_train)]
x_val_glove = [text_to_vec(embeddings_index, x, stopwords, punct, remove_stopwords=True) for x in tqdm(x_val)]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1600/1600 [00:00<00:00, 6375.64it/s]


In [11]:
x_train_glove = np.array(x_train_glove)
x_val_glove = np.array(x_val_glove)

## Base models

In [12]:
logreg = LogisticRegression()

logreg.fit(x_train_glove, y_train)

y_pred = logreg.predict(x_val_glove)
print("Accuracy:", accuracy_score(y_val, y_pred))

Accuracy: 0.558125


In [13]:
xgb = XGBClassifier(
    max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8, nthread=10, 
    learning_rate=0.1, use_label_encoder=False, eval_metric="logloss", random_state=42
)

xgb.fit(x_train_glove, y_train)

y_pred = xgb.predict(x_val_glove)
print("Accuracy:", accuracy_score(y_val, y_pred))

Accuracy: 0.56125


## MLP model

In [14]:
# Scale the data before any neural net
sc = StandardScaler()

x_train_glove_scl = sc.fit_transform(x_train_glove)
x_val_glove_scl = sc.transform(x_val_glove)

In [15]:
# We need to binarize the labels for the neural net
y_train_hot = np_utils.to_categorical(y_train)
y_val_hot = np_utils.to_categorical(y_val)

In [17]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation="relu"))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation("softmax"))

# compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
model.fit(
    x_train_glove_scl, y=y_train_hot, batch_size=64, 
    epochs=50, verbose=1, 
    validation_data=(x_val_glove_scl, y_val_hot)
)

Epoch 1/50
100/100 [==============================] - 2s 4ms/step - loss: 0.9017 - accuracy: 0.5089 - val_loss: 0.7190 - val_accuracy: 0.5375
Epoch 2/50
100/100 [==============================] - 0s 2ms/step - loss: 0.7199 - accuracy: 0.5795 - val_loss: 0.7116 - val_accuracy: 0.5256
Epoch 3/50
100/100 [==============================] - 0s 3ms/step - loss: 0.6683 - accuracy: 0.6105 - val_loss: 0.7120 - val_accuracy: 0.5444
Epoch 4/50
100/100 [==============================] - 0s 3ms/step - loss: 0.6403 - accuracy: 0.6339 - val_loss: 0.7229 - val_accuracy: 0.5462
Epoch 5/50
100/100 [==============================] - 0s 3ms/step - loss: 0.6171 - accuracy: 0.6648 - val_loss: 0.7298 - val_accuracy: 0.5362
Epoch 6/50
100/100 [==============================] - 0s 3ms/step - loss: 0.5922 - accuracy: 0.6742 - val_loss: 0.7471 - val_accuracy: 0.5425
Epoch 7/50
100/100 [==============================] - 0s 3ms/step - loss: 0.5656 - accuracy: 0.7122 - val_loss: 0.7633 - val_accuracy: 0.5425
Epoch 

## LSTM model

In [19]:
# Using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(x_train) + list(x_val))
x_train_seq = token.texts_to_sequences(x_train)
x_val_seq = token.texts_to_sequences(x_val)

# Zero pad the sequences
x_train_pad = sequence.pad_sequences(x_train_seq, maxlen=max_len)
x_val_pad = sequence.pad_sequences(x_val_seq, maxlen=max_len)

word_index = token.word_index

In [20]:
# Create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):

    if word in embeddings_index:
        embedding_matrix[i] = embeddings_index[word]
    else:
        np.random.normal(size=300)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28873/28873 [00:00<00:00, 242742.68it/s]


In [21]:
# A simple LSTM and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.8))

model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

2022-04-17 13:34:19.440553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 34648800 exceeds 10% of free system memory.


In [22]:
model.fit(
    x_train_pad, y=y_train_hot, batch_size=512,
    epochs=30, verbose=1, 
    validation_data=(x_val_pad, y_val_hot)
)

Epoch 1/30
13/13 [==============================] - 5s 224ms/step - loss: 0.7042 - accuracy: 0.5128 - val_loss: 0.6889 - val_accuracy: 0.5350
Epoch 2/30
13/13 [==============================] - 3s 195ms/step - loss: 0.6986 - accuracy: 0.5192 - val_loss: 0.6806 - val_accuracy: 0.5663
Epoch 3/30
13/13 [==============================] - 3s 201ms/step - loss: 0.6902 - accuracy: 0.5450 - val_loss: 0.6740 - val_accuracy: 0.5888
Epoch 4/30
13/13 [==============================] - 3s 206ms/step - loss: 0.6810 - accuracy: 0.5730 - val_loss: 0.6650 - val_accuracy: 0.5931
Epoch 5/30
13/13 [==============================] - 3s 204ms/step - loss: 0.6638 - accuracy: 0.6023 - val_loss: 0.6435 - val_accuracy: 0.6331
Epoch 6/30
13/13 [==============================] - 3s 223ms/step - loss: 0.6570 - accuracy: 0.6184 - val_loss: 0.6626 - val_accuracy: 0.5994
Epoch 7/30
13/13 [==============================] - 3s 225ms/step - loss: 0.6418 - accuracy: 0.6363 - val_loss: 0.6339 - val_accuracy: 0.6388
Epoch 

## Bidirectional LSTMs

In [24]:
# A simple bidirectional LSTM and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(100, activation="relu"))
model.add(Dropout(0.8))

model.add(Dense(100, activation="relu"))
model.add(Dropout(0.8))

model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=0, mode="auto")

2022-04-17 13:35:46.519582: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 34648800 exceeds 10% of free system memory.


In [25]:
model.fit(
    x_train_pad, y=y_train, batch_size=512, 
    epochs=100, verbose=1, 
    validation_data=(x_val_pad, y_val), callbacks=[earlystop]
)

Epoch 1/100
13/13 [==============================] - 10s 420ms/step - loss: 0.7157 - accuracy: 0.4975 - val_loss: 0.6915 - val_accuracy: 0.5031
Epoch 2/100
13/13 [==============================] - 7s 518ms/step - loss: 0.7015 - accuracy: 0.5080 - val_loss: 0.6920 - val_accuracy: 0.5206
Epoch 3/100
13/13 [==============================] - 7s 498ms/step - loss: 0.6959 - accuracy: 0.5138 - val_loss: 0.6913 - val_accuracy: 0.5200
Epoch 4/100
13/13 [==============================] - 5s 411ms/step - loss: 0.6958 - accuracy: 0.5033 - val_loss: 0.6908 - val_accuracy: 0.5562
Epoch 5/100
13/13 [==============================] - 5s 395ms/step - loss: 0.6944 - accuracy: 0.5058 - val_loss: 0.6909 - val_accuracy: 0.5581
Epoch 6/100
13/13 [==============================] - 5s 387ms/step - loss: 0.6887 - accuracy: 0.5295 - val_loss: 0.6888 - val_accuracy: 0.5625
Epoch 7/100
13/13 [==============================] - 5s 413ms/step - loss: 0.6892 - accuracy: 0.5298 - val_loss: 0.6864 - val_accuracy: 0.563

In [26]:
# GRU and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(512, activation="relu"))
model.add(Dropout(0.8))

model.add(Dense(512, activation="relu"))
model.add(Dropout(0.8))

model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=0, mode="auto")

2022-04-17 13:42:23.300596: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 34648800 exceeds 10% of free system memory.


In [27]:
model.fit(
    x_train_pad, y=y_train, 
    batch_size=512, epochs=100, 
    verbose=1, validation_data=(x_val_pad, y_val), callbacks=[earlystop]
)

Epoch 1/100
13/13 [==============================] - 8s 374ms/step - loss: 0.7148 - accuracy: 0.4922 - val_loss: 0.6913 - val_accuracy: 0.5206
Epoch 2/100
13/13 [==============================] - 5s 365ms/step - loss: 0.7028 - accuracy: 0.5108 - val_loss: 0.6868 - val_accuracy: 0.5506
Epoch 3/100
13/13 [==============================] - 5s 409ms/step - loss: 0.6944 - accuracy: 0.5339 - val_loss: 0.6805 - val_accuracy: 0.5706
Epoch 4/100
13/13 [==============================] - 5s 353ms/step - loss: 0.6873 - accuracy: 0.5609 - val_loss: 0.6713 - val_accuracy: 0.5894
Epoch 5/100
13/13 [==============================] - 5s 360ms/step - loss: 0.6837 - accuracy: 0.5761 - val_loss: 0.6760 - val_accuracy: 0.5844
Epoch 6/100
13/13 [==============================] - 5s 382ms/step - loss: 0.6752 - accuracy: 0.5742 - val_loss: 0.6724 - val_accuracy: 0.5719
Epoch 7/100
13/13 [==============================] - 5s 407ms/step - loss: 0.6732 - accuracy: 0.5863 - val_loss: 0.6589 - val_accuracy: 0.6388